# Progetto Computer Science

Edwin Casiraghi e Stefano Fiorini

In [241]:
import pandas as pd 
import ast 
import json
from urllib.parse import urlparse

Caricare i due dataset

In [277]:
movies = pd.read_csv('tmdb_5000_movies.csv')
df_movies = movies.rename(columns = {'id' : 'movie_id'})
credits = pd.read_csv('tmdb_5000_credits.csv')
display(df_movies.head(4), credits.head(6))

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.000000,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.200000,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.000000,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.900000,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.000000,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.300000,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.000000,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.600000,9106


,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
5,559,Spider-Man 3,"[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de..."


Funzione estrazione delle colonne json

In [243]:
def estrazione_json(db, column):
    result = db[column].apply(pd.read_json)
    movie_id = db['movie_id'] 
    for i in range(len(db[column])):
        result[i]['movie_id'] = movie_id[i]
    return result.pipe(lambda x: pd.concat(x.values))

Dataset estratti

In [245]:
cast = estrazione_json(credits, 'cast')
movies_keyword = estrazione_json(df_movies, 'keywords')
movies_genere = estrazione_json(df_movies, 'genres')
movies_aziende = estrazione_json(df_movies, 'production_companies')
production_nazioni = estrazione_json(df_movies, 'production_countries').rename(columns = {'iso_3166_1' : 'id'})
lingue_parlate_movies = estrazione_json(df_movies, 'spoken_languages')
crew = estrazione_json(credits, 'crew')

Modifica dell'attributo gender

In [246]:
data = {'gender': [0.00,1.00,2.00],
        'Gender': ['Nan','F','M']}
df = pd.DataFrame(data)
cast_normalizzato = pd.merge(cast, df, on = 'gender').drop(['gender'], axis = 1)
crew_normalizzato = pd.merge(crew, df, on = 'gender').drop(['gender'], axis = 1)

Creazione di una colonna con solo l'anno

In [247]:
df_movies['year'] = pd.to_datetime(df_movies['release_date'],format='%Y-%m-%d').dt.year

1) For each movie, compute the number of cast members.

In [248]:
cast_1 = cast.groupby('movie_id').size().to_frame().reset_index().rename(columns={0: 'cast'})
numero_cast = pd.merge(cast_1, credits[['title', 'movie_id']], on = 'movie_id')
numero_cast.head(6)

,movie_id,cast,title
0,5,24,Four Rooms
1,11,106,Star Wars
2,12,24,Finding Nemo
3,13,66,Forrest Gump
4,14,40,American Beauty
5,16,29,Dancer in the Dark


2) How many movies do not have a homepage? 

In [278]:
movies_without_homepage = df_movies[df_movies['homepage'].isnull()]
len(movies_without_homepage)

3091

3) For each year, how many movies do not have a homepage?

In [250]:
movies_without_homepage_size = movies_without_homepage.groupby('year').size().to_frame().reset_index().rename(columns = {0 : 'dimensioni'}).head(6)
movies_without_homepage_size['year'] = movies_without_homepage_size['year'].astype(int)
movies_without_homepage_size.head(6)

,year,dimensioni
0,1916,1
1,1925,1
2,1927,1
3,1929,2
4,1930,1
5,1932,1


4) Extract the domain of each homepage

In [290]:
dt = movies[movies['homepage'].notnull()].reset_index().drop('index', axis = 1)
domain = []
for i in range(len(dt)):
    df_parse = urlparse(dt.homepage[i])
    domain.append(df_parse)
domain_1 = pd.DataFrame(domain).netloc.str.split('.')
result = []
for i in range(len(dt)):
    domain_1_parse = domain_1[i][-1]
    result.append(domain_1_parse)
result_1 = pd.DataFrame(result)
result_1 = pd.concat([dt[['original_title', 'homepage']], result_1], axis=1).rename(columns = {0 : 'domain'})
result_1.head(6)

,original_title,homepage,domain
0,Avatar,http://www.avatarmovie.com/,com
1,Pirates of the Caribbean: At World's End,http://disney.go.com/disneypictures/pirates/,com
2,Spectre,http://www.sonypictures.com/movies/spectre/,com
3,The Dark Knight Rises,http://www.thedarkknightrises.com/,com
4,John Carter,http://movies.disney.com/john-carter,com
5,Spider-Man 3,http://www.sonypictures.com/movies/spider-man3/,com


5) Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary)

In [252]:
keywords = movies_keyword[["id", "name"]].drop_duplicates()
keywords.id = keywords.id.astype(int)
keywords_name = keywords.set_index("id")
keywords_movies = movies_keyword[["id","movie_id"]].reset_index().drop('index', axis = 1)
keywords_movies.columns = ["keyword_id", "movie_id"]
keywords_movies.keyword_id = movies_keyword_1.keyword_id.astype(int)
display(keywords_movies.head(4), keywords_name.head(4))

,keyword_id,movie_id
0,1463,19995
1,2964,19995
2,3386,19995
3,3388,19995


,name
id,
1463,culture clash
2964,future
3386,space war
3388,space colony


In [253]:
genere = movies_genere[["id", "name"]].drop_duplicates()
genere.id = genere.id.astype(int)
genere_name = genere.set_index("id")
genere_movies = movies_genere[["id","movie_id"]].reset_index().drop('index', axis = 1).rename(columns = {'id':'genere_id'})
genere_movies.genere_id = genere_movies.genere_id.astype(int)
display(genere_movies.head(6), genere_name.head(6))

,genere_id,movie_id
0,28,19995
1,12,19995
2,14,19995
3,878,19995
4,12,285
5,14,285


,name
id,
28,Action
12,Adventure
14,Fantasy
878,Science Fiction
80,Crime
18,Drama


In [254]:
aziende = movies_aziende[['id', 'name']].drop_duplicates()
aziende.id = aziende.id.astype(int)
aziende_name = aziende.set_index('id')
aziende_movies = movies_aziende[['id', 'movie_id']].reset_index().drop('index', axis = 1).rename(columns = {'id':'aziende_id'})
aziende_movies.aziende_id = aziende_movies.aziende_id.astype(int)
display(aziende_movies.head(6), aziende_name.head(6))

,aziende_id,movie_id
0,289,19995
1,306,19995
2,444,19995
3,574,19995
4,2,285
5,130,285


,name
id,
289,Ingenious Film Partners
306,Twentieth Century Fox Film Corporation
444,Dune Entertainment
574,Lightstorm Entertainment
2,Walt Disney Pictures
130,Jerry Bruckheimer Films


In [255]:
nazioni = production_nazioni[['id', 'name']].drop_duplicates()
nazioni_name = nazioni.set_index('id')
nazioni_production = production_nazioni[['id', 'movie_id']].reset_index().drop('index', axis = 1).rename(columns = {'id':'nazioni_id'})
display(nazioni_production.head(6), nazioni_name.head(6))

,nazioni_id,movie_id
0,US,19995
1,GB,19995
2,US,285
3,GB,206647
4,US,206647
5,US,49026


,name
id,
US,United States of America
GB,United Kingdom
JM,Jamaica
BS,Bahamas
DM,Dominica
CZ,Czech Republic


In [256]:
lingue_movies = lingue_parlate_movies.rename(columns = {'iso_639_1' : 'id'})
lingue_name = lingue_movies[['id', 'name']].drop_duplicates()
lingue_name_1 = lingue_parlate.set_index('id')
lingue_movies = lingue_movies[['id', 'movie_id']].reset_index().drop('index', axis = 1).rename(columns = {'id': 'lingua_id'})
display(lingue_movies.head(6), lingue_name_1.head(6))

,lingua_id,movie_id
0,en,19995
1,es,19995
2,en,285
3,fr,206647
4,en,206647
5,es,206647


,name
id,
en,English
es,Español
fr,Français
it,Italiano
de,Deutsch
tr,Türkçe


In [257]:
cast_normalizzato.id = cast_normalizzato.id.astype(int)
cast_normalizzato.cast_id = cast_normalizzato.cast_id.astype(int)
cast_normalizzato.order = cast_normalizzato.order.astype(int)
attori = cast_normalizzato[['id', 'name', 'Gender']]
attori_credito = cast_normalizzato[['cast_id', 'character', 'credit_id', 'id', 'order']].set_index('credit_id')
display(attori.head(6), attori_credito.head(6))

,id,name,Gender
0,65731,Sam Worthington,M
1,32747,Stephen Lang,M
2,1771,Giovanni Ribisi,M
3,59231,Joel David Moore,M
4,15853,Wes Studi,M
5,10964,Laz Alonso,M


,cast_id,character,id,order
credit_id,,,,
5602a8a7c3a3685532001c9a,242,Jake Sully,65731,0
52fe48009251416c750ac9cf,4,Col. Quaritch,32747,3
52fe48009251416c750ac9e1,8,Selfridge,1771,5
52fe48009251416c750ac9dd,7,Norm Spellman,59231,6
52fe48009251416c750ac9ed,11,Eytukan,15853,8
52fe48009251416c750ac9e9,10,Tsu'Tey,10964,9


In [258]:
crew_normalizzato.id = crew_normalizzato.id.astype(int)
lavoratori = crew_normalizzato[['id', 'name', 'Gender']]
lavoratori_credito = crew_normalizzato[['id', 'department', 'credit_id', 'job', 'movie_id']].set_index('credit_id')
display(lavoratori.head(6), lavoratori_credito.head(6))

,id,name,Gender
0,1721,Stephen E. Rivkin,Nan
1,900,Christopher Boyes,Nan
2,900,Christopher Boyes,Nan
3,6690,Jill Brooks,Nan
4,6883,Dick Bernstein,Nan
5,8159,Shannon Mills,Nan


,id,department,job,movie_id
credit_id,,,,
52fe48009251416c750aca23,1721,Editing,Editor,19995
54491c89c3a3680fb4001cf7,900,Sound,Sound Designer,19995
54491cb70e0a267480001bd0,900,Sound,Supervising Sound Editor,19995
549598c3c3a3686ae9004383,6690,Visual Effects,Visual Effects Producer,19995
5495a0fac3a3686ae9004468,6883,Sound,Music Editor,19995
54959706c3a3686af3003e81,8159,Sound,Sound Effects Editor,19995


In [259]:
personale = pd.concat([attori, lavoratori], axis = 0).drop_duplicates().set_index('id')
personale.head(6)

,name,Gender
id,,
65731,Sam Worthington,M
32747,Stephen Lang,M
1771,Giovanni Ribisi,M
59231,Joel David Moore,M
15853,Wes Studi,M
10964,Laz Alonso,M


In [260]:
df_movies_normalizzato = df_movies.drop(['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages'], axis = 1).set_index('movie_id')
df_movies_normalizzato.head(4)

,budget,homepage,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,year
movie_id,,,,,,,,,,,,,,,
19995,237000000,http://www.avatarmovie.com/,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,2787965087,162.000000,Released,Enter the World of Pandora.,Avatar,7.200000,11800,2009.000000
285,300000000,http://disney.go.com/disneypictures/pirates/,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,961000000,169.000000,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.900000,4500,2007.000000
206647,245000000,http://www.sonypictures.com/movies/spectre/,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,880674609,148.000000,Released,A Plan No One Escapes,Spectre,6.300000,4466,2015.000000
49026,250000000,http://www.thedarkknightrises.com/,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,2012-07-16,1084939099,165.000000,Released,The Legend Ends,The Dark Knight Rises,7.600000,9106,2012.000000


In [261]:
credits_normalizzato = credits.drop(['cast', 'crew'], axis = 1).set_index('movie_id')
credits_normalizzato.head(6)

,title
movie_id,
19995,Avatar
285,Pirates of the Caribbean: At World's End
206647,Spectre
49026,The Dark Knight Rises
49529,John Carter
559,Spider-Man 3


6) For each movie, compute the gross margin (difference between revenue and budget).

In [262]:
df_movies['gross_margin'] = df_movies.revenue - movies.budget 
df_movies[['title','revenue','budget','gross_margin']].head(6)

,title,revenue,budget,gross_margin
0,Avatar,2787965087,237000000,2550965087
1,Pirates of the Caribbean: At World's End,961000000,300000000,661000000
2,Spectre,880674609,245000000,635674609
3,The Dark Knight Rises,1084939099,250000000,834939099
4,John Carter,284139100,260000000,24139100
5,Spider-Man 3,890871626,258000000,632871626


7) For each movie, compute the number of crew members.

In [292]:
crew_1 = crew.groupby('movie_id').size()
crew_2 = pd.DataFrame(crew_1).reset_index().rename(columns={0: 'lavoratori'})
numero_lavoratori = pd.merge(crew_2, credits[['title', 'movie_id']], on = 'movie_id')
numero_lavoratori.head(6)

,movie_id,lavoratori,title
0,5,88,Four Rooms
1,11,20,Star Wars
2,12,104,Finding Nemo
3,13,93,Forrest Gump
4,14,109,American Beauty
5,16,62,Dancer in the Dark


8) For each movie, compute the number of directors.

In [264]:
direttori = crew[crew['job'] == 'Director'].groupby('movie_id').size().to_frame().reset_index().rename(columns={0: 'Direttori'})
numero_direttori = pd.merge(direttori, credits[['title', 'movie_id']], on = 'movie_id')
numero_direttori.head(6)


,movie_id,Direttori,title
0,5,4,Four Rooms
1,11,1,Star Wars
2,12,1,Finding Nemo
3,13,1,Forrest Gump
4,14,1,American Beauty
5,16,1,Dancer in the Dark


9) For each language, compute the number of movies where such language is spoken.

In [293]:
lingua = lingue_parlate_movies.groupby( 'iso_639_1').size().to_frame().reset_index().rename(columns={0: 'numero film'})
lingua.head(6)

,iso_639_1,numero film
0,af,7
1,am,1
2,ar,67
3,bg,3
4,bm,1
5,bn,1


10) For each company and each decade, compute the overall revenue.

In [266]:
movies_1 = df_movies[df_movies['year'].notnull()]
movies_1['decade'] = (movies_1['year'] / 10).astype(int)
company = pd.merge(movies_aziende, movies_1[['decade', 'title', 'movie_id', 'revenue', 'year']]).drop('id', axis = 1)
company_1 = company.groupby(['decade', 'name'])['revenue'].sum().to_frame().reset_index()
company_1.head(6)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,decade,name,revenue
0,191,Triangle Film Corporation,8394751
1,191,Wark Producing Corp.,8394751
2,192,Metro-Goldwyn-Mayer (MGM),26358000
3,192,Nero Films,0
4,192,Paramount Pictures,650422
5,192,Universum Film (UFA),650422


11) For each decade, compute the company with maximum revenue

In [303]:
best_company = company_1[company_1.groupby(['decade'])['revenue'].transform(max) == company_1['revenue']]
best_company

,decade,name,revenue
0,191,Triangle Film Corporation,8394751
1,191,Wark Producing Corp.,8394751
2,192,Metro-Goldwyn-Mayer (MGM),26358000
10,193,Metro-Goldwyn-Mayer (MGM),400176459
13,193,Selznick International Pictures,400176459
37,194,Walt Disney Productions,351747150
50,195,Metro-Goldwyn-Mayer (MGM),73770324
82,196,Eon Productions,598134765
248,197,United Artists,1392271577
450,198,Paramount Pictures,4431144486


12) In each year, how many movies have revenue smaller than the budget?

In [268]:
movies_revenue = df_movies[df_movies["gross_margin"] < 0].groupby(by = ("year")).size().to_frame().reset_index().rename(columns = {0 : 'numero film'})
movies_revenue['year'] = movies_revenue['year'].astype(int)
movies_revenue.head(6)

,year,numero film
0,1927,1
1,1947,1
2,1948,1
3,1954,1
4,1962,1
5,1963,1


13) Distribute the revenue according to the order of appearance in a movie. Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue

In [270]:
protagonisti = pd.merge(cast[['movie_id', 'name', 'order']], movies_1[['revenue', 'movie_id']], on = 'movie_id')
protagonisti['retribuzione'] = (protagonisti['revenue']/2)/(2**protagonisti['order'])
pd.set_option('display.float_format', lambda x: '%.6f' % x)
protagonisti['order'] = protagonisti['order'].astype(int)
protagonisti[protagonisti['movie_id'] == 49026].head(6)

,movie_id,name,order,revenue,retribuzione
200,49026,Christian Bale,0,1084939099,542469549.500000
201,49026,Michael Caine,1,1084939099,271234774.750000
202,49026,Gary Oldman,2,1084939099,135617387.375000
203,49026,Anne Hathaway,3,1084939099,67808693.687500
204,49026,Tom Hardy,4,1084939099,33904346.843750
205,49026,Marion Cotillard,5,1084939099,16952173.421875


In [271]:
sum(protagonisti[protagonisti.movie_id==49026].retribuzione)

1084939098.9999998

In [272]:
df_movies.revenue[3]

1084939099

14) For each actor find the total revenue attributed to him/her

In [304]:
totale_retribuzione_attori = protagonisti.groupby('name').sum().reset_index().sort_values(by = 'retribuzione', ascending = False).drop(['revenue', 'order'], axis = 1)
totale_retribuzione_attori.head(6)

,name,movie_id,retribuzione
50772,Tom Cruise,775502,3976017174.373529
50821,Tom Hanks,893665,3799897896.061697
43611,Robert Downey Jr.,1031283,3631679406.093750
25844,Johnny Depp,2056384,3511502683.804348
53132,Will Smith,1473901,3281555344.087485
4872,Ben Stiller,1401034,2832962893.159470


15) Find the actor that is responsible for the most overall revenue

In [315]:
totale_retribuzione_attori.loc[totale_retribuzione_attori['retribuzione'].idxmax()]


name                  Tom Cruise
movie_id                  775502
retribuzione   3976017174.373529
Name: 50772, dtype: object